# Create Lists of Admin1

**[Work in progress]**

This notebook creates countries for ingestion into a Knowledge Graph.

Data source: [GeoNames.org](https://download.geonames.org/export/dump/)

Author: Peter Rose (pwrose@ucsd.edu)

In [1]:
import os
from pathlib import Path
import pandas as pd

In [2]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

In [3]:
NEO4J_HOME = Path(os.getenv('NEO4J_HOME'))
print(NEO4J_HOME)

/Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-4af96121-2328-4e2f-ba60-6d8b728a26d5/installation-4.0.3


### Create admin1

In [4]:
admin1_url = 'https://download.geonames.org/export/dump/admin1CodesASCII.txt'

In [5]:
names = ['code', 'name', 'name_ascii', 'geonameid']

In [6]:
admin1 = pd.read_csv(admin1_url, sep='\t', dtype='str', names=names)
admin1 = admin1[['code', 'name_ascii']]

### Standardize column names for Knowlege Graph
* id: unique identifier for country
* name: name of node
* parentId: unique identifier for continent
* properties: camelCase

In [7]:
admin1.rename(columns={'code': 'id'}, inplace=True) # standard id column to link nodes
admin1.rename(columns={'name_ascii': 'name'}, inplace=True)
admin1['code'] = admin1['id'].str.split('.', expand=True)[1]
admin1['parentId'] = admin1['id'].str.split('.', expand=True)[0]

### Example

In [8]:
admin1.query("name == 'California'")

,id,name,code,parentId
3712,US.CA,California,CA,US


### Export a minimum subset for now

In [9]:
admin1 = admin1[['id','name','code','parentId']]
admin1 = admin1.fillna('')

In [10]:
admin1.head()

,id,name,code,parentId
0,AD.06,Sant Julia de Loria,06,AD
1,AD.05,Ordino,05,AD
2,AD.04,La Massana,04,AD
3,AD.03,Encamp,03,AD
4,AD.02,Canillo,02,AD


In [11]:
admin1.to_csv(NEO4J_HOME / "import/00f-GeoNamesAdmin1.csv", index=False)